In [1]:
import pandas as pd
import re
from tqdm import tqdm 

In [2]:
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher

# Prize winners in HathiTrust titlemeta.tsv

In [3]:
"""
let's open our data
+
keep only the columns with which we gonna work
"""

titlemeta = pd.read_csv("titlemeta.tsv", sep="\t", low_memory=False)
titlemeta_small = titlemeta[['author', 'title', 'docid']].drop_duplicates(['author', 'title']).reset_index(drop=True)

prizes = pd.read_csv("prizes.tsv", sep="\t", low_memory=False)

In [4]:
"""
keep only the columns with which we gonna work
"""

titlemeta_small.shape

(123371, 3)

In [5]:
"""
clean the titles from everything what follows the mojibake '| $c:' 
+
let's crerate a new column with clean titles
"""

clean_titles = []
for index, row in titlemeta_small.iterrows():
    if re.findall(" \| \$c:", row["title"]):
        slice_index = row["title"].rfind("| $c:")
        clean_title = re.sub("[\/\|\[\]]", '', row["title"][:slice_index])
        clean_titles.append(clean_title)
    else:
        clean_title = re.sub("[\/\|\[\]]", '', row["title"][:slice_index])
        clean_titles.append(clean_title)

        
titlemeta_small["clean_title"] = clean_titles
        

In [6]:
titlemeta_small

author  \
0                   Spencer, Louise Reid   
1                        Baker, Robert H   
2                       Dickens, Charles   
3                        Stretton, Hesba   
4                        Stretton, Hesba   
...                                  ...   
123366                               NaN   
123367  Wison, Steven E. (Steven Eugene)   
123368                    Jeon, Heecheon   
123369     Wells, H. G. (Herbert George)   
123370                     Davis, John A   

                                                    title  \
0          Guerrilla wife | $c: [by] Louise Reid Spencer.   
1       The suburbs : | a novel / | $c: by Robert H. B...   
2       Edwin Drood. | $c: By Charles Dickens. With il...   
3                        Carola, | $c: by Hesba Stretton.   
4               In prison & out. | $c: By Hesba Stretton.   
...                                                   ...   
123366  Six plays of the yiddish theatre / | $c: [tran...   
123367  The ghosts of Anatolia : | an epic journey to ...   
123368  Subjectivity of différance : | a poiesis of de...   
123369   Certain personal matters / | $c: by H. G. Wells.   
123370  Qing guo nu li : | fu ren Zhongguo fu nü sheng...   

                      docid                                        clean_title  
0        mdp.39015031913893                                    Guerrilla wife   
1        mdp.39015003936864                           The suburbs :  a novel    
2        mdp.39015068342305                                      Edwin Drood.   
3        mdp.39015055066586                                           Carola,   
4        mdp.39015055066594                                  In prison & out.   
...                     ...                                                ...  
123366  umn.31951d030369342                 Six plays of the yiddish theatre    
123367   mdp.39076002906829  The ghosts of Anatolia :  an epic journey to f...  
123368   inu.30000127716409  Subjectivity of différance :  a poiesis of dec...  
123369   ien.35556042103887                         Certain personal matters    
123370   mdp.39015092163339  Qing guo nu li :  fu ren Zhongguo fu nü sheng ...  

[123371 rows x 4 columns]

In [7]:
"""
let's do the fuzzy matching for the HT dataframe and another df
+
create a new dataframe with the intersections
NB: the code is partly borrowed from Wenyi's notebook
"""

def dataframes_match(titlemeta, df_to_compare, df_author, df_title, df_date,
                    n_ratio_author, n_ratio_title):

    titlemeta_ID = []
    titlemeta_author = []
    titlemeta_title = []
    prize_author = []
    prize_title = []
    prize_date = []
    ratio_author_lst = []
    ratio_title_lst = []

    """
    perhaps, it's better to create another algorythm, loop in loop works very slowly -> maybe lambda can 
    work better
    """

    for idx, row in tqdm(titlemeta.iterrows()):

        author = str(row["author"]).lower()
        title = str(row["clean_title"]).lower()

        if title.startswith('the '):        
            title = title[4:]               


        for idx2, row2 in df_to_compare.iterrows():
            author2 = str(row2["author"]).lower()
            title2 = str(row2["title"]).lower()

            if title2.startswith('the '):
                title2 = title2[4:]

            ratio_author = fuzz.ratio(author2, author)
            ratio_title = fuzz.ratio(title2, title)

            if ratio_author > n_ratio_author and ratio_title > n_ratio_title:


                titlemeta_ID.append(row["docid"])
                titlemeta_author.append(row["author"])
                titlemeta_title.append(row["clean_title"])
                prize_author.append(row2["author"])
                prize_title.append(row2["title"])
                prize_date.append(row2["prizedate"])
                ratio_author_lst.append(ratio_author)
                ratio_title_lst.append(ratio_title)

            else:
                continue

    final_df = pd.DataFrame(list(zip(titlemeta_ID, titlemeta_author, titlemeta_title, prize_author,
                                      prize_title, prize_date, ratio_author_lst, ratio_title_lst)), 
                            columns=["titlemeta_ID", "titlemeta_author", "titlemeta_title", 
                                    df_author, df_title, df_date, "ratio_author", "ratio_title"])
    return final_df

In [8]:
hathi_prize = dataframes_match(titlemeta_small, prizes, "prize_author", "prize_title", "prize_date",
                               n_ratio_author = 75, n_ratio_title = 0)

123371it [12:43, 161.68it/s]


In [9]:
"""
let's save our data to csv file
"""

hathi_prize.to_csv('list_of_matches_HT_prize_75_0.csv', sep = '\t', index = False)

In [10]:
"""
let's filter two types of authors: those, who got the Prize for the whole work and those, who got the
Prize for a particular work. The ratio of the title is important for us. 
Perhaphs, the ratio_title should be lower than 50, I set it just according to my very subjective opinion
"""
any_book = hathi_prize[hathi_prize['prize_title'] == '#matchall'].reset_index(drop=True)
specific_books = hathi_prize[hathi_prize['prize_title'] != '#matchall'].reset_index(drop=True)
specific_books = specific_books[specific_books['ratio_title'] > 50].reset_index(drop=True)

In [11]:
any_book

titlemeta_ID   titlemeta_author  \
0              wu.89099437782    France, Anatole   
1          nyp.33433074955463     Mozeen, Thomas   
2          mdp.39015010783762  Hawkesworth, John   
3    uc2.ark+=13960=t7vm44d7m  Hawkesworth, John   
4          mdp.39015063891314     Leland, Thomas   
..                        ...                ...   
552        mdp.39015069362567     Mallon, Thomas   
553        mdp.39015079234749    Maltman, Thomas   
554        inu.30000095637116    Lewis, Sinclair   
555        mdp.39015080827390      Emson, Thomas   
556        mdp.39015078776195   Lashner, William   

                                       titlemeta_title       prize_author  \
0    The authorized English translations of the nov...    France, Anatole   
1                                       Young Scarron        Mann, Thomas   
2    Almoran and Hamet:  an Oriental tale. In two v...   Galsworthy, John   
3                                    Almoran and Hamet   Galsworthy, John   
4    Longsword, Earl of Salisbury.  An historical r...       Mann, Thomas   
..                                                 ...                ...   
552                                 Fellow travelers         Mann, Thomas   
553                                  The night birds         Mann, Thomas   
554  The short stories of Sinclair Lewis (1904-1949)      Lewis, Sinclair   
555                                          Skarlet         Mann, Thomas   
556                                   Blood and bone    Faulkner, William   

    prize_title  prize_date  ratio_author  ratio_title  
0     #matchall        1921           100           13  
1     #matchall        1929            77           17  
2     #matchall        1932            79           19  
3     #matchall        1932            79           31  
4     #matchall        1929            77           16  
..          ...         ...           ...          ...  
552   #matchall        1929            85           22  
553   #matchall        1929            81            9  
554   #matchall        1930           100           15  
555   #matchall        1929            80           22  
556   #matchall        1949            79            8  

[557 rows x 8 columns]

In [12]:
"""
let's recheck the intersections between titles and get rid off books which are not in the prize_title column
"""

for index, row in specific_books.iterrows():
    if str(row["prize_title"]).lower() not in str(row["titlemeta_title"]).lower():
        specific_books.drop(index, inplace=True)

In [13]:
"""
let's concat two our dataframes and save them as an independenct .csv file
"""
prize_winners = pd.concat([any_book, specific_books]).reset_index(drop=True)
prize_winners.to_csv('match_prize_winners_75.csv', index = False)

# Bestsellers in HathiTrust titlemeta.tsv

In [14]:
bestsellers = pd.read_csv("new_unsworth_bestsellers.csv", low_memory=False)

In [15]:
bestsellers

year              author                title
0     1900      Johnston, Mary  To Have and To Hold
1     1900  Cholmondeley, Mary          Red Pottage
2     1900       Grant, Robert     Unleavened Bread
3     1900   Allen, James Lane     The Reign of Law
4     1900   Bacheller, Irving          Eben Holden
...    ...                 ...                  ...
1116  1999        Fitch, Janet       White Oleander
1117  1999    Sparks, Nicholas   A Walk to Remember
1118  1999    Patterson, James  Pop Goes the Weasel
1119  1999  Cornwell, Patricia         Black Notice
1120  1999     Steel, Danielle           Granny Dan

[1121 rows x 3 columns]

In [16]:
bestsellers.rename(columns = {'year':'prizedate'}, inplace = True)

In [17]:
hathi_bestsellers = dataframes_match(titlemeta_small, bestsellers, "bestseller_author", "bestseller_title",
                               "bestseller_year", n_ratio_author = 75, n_ratio_title = 50)

123371it [4:22:59,  7.82it/s]


In [18]:
hathi_bestsellers.to_csv('list_of_matches_HT_bestsellers_75_50.csv', index = False)

In [19]:
hathi_bestsellers[hathi_bestsellers['ratio_title'] > 55].reset_index(drop=True)
# specific_books[specific_books['ratio_title'] > 50].reset_index(drop=True)

titlemeta_ID         titlemeta_author  \
0          mdp.39015063548997             Glyn, Elinor   
1    loc.ark+=13960=t3st7r64d              Greene, Asa   
2    uc2.ark+=13960=t14m9612t            Stewart, E. M   
3                uc1.b3962029  Churchill, Winston, Sir   
4          nyp.33433082272703       Cholmondeley, Mary   
..                        ...                      ...   
876        inu.30000100635840      Beauvoir, Simone de   
877          pst.000065617298               Grey, Zane   
878        mdp.39015064953329         Bachman, Richard   
879        mdp.39015070756450           Doctorow, E. L   
880        inu.30000124485677            Walters, Mike   

            titlemeta_title    bestseller_author         bestseller_title  \
0      Visits of Elizabeth          Glyn, Elinor  The Visits of Elizabeth   
1               Travels in        Greene, Graham     Travels with My Aunt   
2          Lillias Davenant        Stewart, Mary     The Last Enchantment   
3          Richard Carvel,    Churchill, Winston           Richard Carvel   
4             Red pottage,    Cholmondeley, Mary              Red Pottage   
..                      ...                  ...                      ...   
876         The mandarins    de Beauvoir, Simone            The Mandarins   
877       To the last man             Grey, Zane          To the Last Man   
878      Blaze :  a novel          Bach, Richard             One: A Novel   
879  Loon Lake :  a novel         Doctorow, E.L.                Loon Lake   
880         The adversary          Waltari, Mika           The Adventurer   

     bestseller_year  ratio_author  ratio_title  
0               1901           100           97  
1               1970            80           65  
2               1979            77           56  
3               1900            88           93  
4               1900           100           92  
..               ...           ...          ...  
876             1956            84           90  
877             1922           100           94  
878             1988            90           67  
879             1980            93           58  
880             1950            77           57  

[881 rows x 8 columns]